In [41]:
import tkinter as tk
from tkinter import filedialog, Canvas, Toplevel, Scale
from PIL import Image, ImageTk, ImageDraw
import numpy as np
import tensorflow as tf
import cv2
import keras.backend as K
from keras.models import load_model
from metrics import f1, iou, padder, predict_image
from matplotlib import pyplot as plt
from patchify import patchify,unpatchify

class ImageAnnotationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Annotation App")

        self.canvas_frame = tk.Frame(root)
        self.canvas_frame.pack(fill=tk.BOTH, expand=True, anchor=tk.NW)

        self.button_frame = tk.Frame(root)
        self.button_frame.pack(side=tk.TOP, padx=10, pady=10, anchor=tk.NW)

        self.button = tk.Button(self.button_frame, text="Open Image", command=self.open_image)
        self.button.pack(side=tk.RIGHT)

        self.canvas = Canvas(self.canvas_frame)
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        self.rect = None
        self.start_x = None
        self.start_y = None
        # Load your Keras model
        self.model = load_model("model/root_4_ss.h5", custom_objects={'f1': f1,'iou':iou})
          


    def open_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.image = Image.open(file_path)
            self.original_image = self.image.copy()  # Keep a copy of the original image
 
            # Calculate the aspect ratio of the original image
            aspect_ratio = self.image.width / self.image.height
            print(aspect_ratio)

            new_width = 1080
            new_height = 1080
            self.change_ratio = self.image.width / new_width
            # Resize the image with the new dimensions
            self.image = self.image.resize((new_width, new_height))
            self.photo = ImageTk.PhotoImage(self.image)

            self.canvas.config(scrollregion=(0, 0, new_width, new_height))
            self.canvas.create_image(0, 0, image=self.photo, anchor=NW)
            self.canvas.bind("<ButtonPress-1>", self.on_button_press)
            self.canvas.bind("<B1-Motion>", self.on_move_press)
            self.canvas.bind("<ButtonRelease-1>", self.on_button_release)


    def on_button_press(self, event):
        self.start_x = event.x
        self.start_y = event.y
        if self.rect:
            self.canvas.delete(self.rect)

        self.rect = self.canvas.create_rectangle(self.start_x, self.start_y, 1, 1, outline="red")

    def on_move_press(self, event):
        curX, curY = (event.x, event.y)
        self.canvas.coords(self.rect, self.start_x, self.start_y, curX, curY)

    def on_button_release(self, event):
        # Extract the region within the rectangle
        x0, y0, x1, y1 = self.canvas.coords(self.rect)
        print(x0, x1, y0, y1)
        x0 *= self.change_ratio
        x1 *= self.change_ratio
        y0 *= self.change_ratio
        y1 *= self.change_ratio
        print(x0, x1, y0, y1)
        bbox = (int(x0), int(y0), int(x1), int(y1))
        cropped_image = self.original_image.crop(bbox)
        cropped_image = np.asarray(cropped_image) / 255.0
        prediction = predict_image(cropped_image, self.model)
        

        # Convert the prediction result to an image
        plt.imshow(prediction)  # Assuming prediction is a 2D array representing an image
        plt.axis('off')
        plt.savefig("prediction.png")
        plt.close()

        # Open a new window to display the prediction image
        prediction_window = Toplevel(self.root)
        prediction_window.title("Prediction Result")
        prediction_image = Image.open("prediction.png")
        prediction_photo = ImageTk.PhotoImage(prediction_image)
        prediction_label = tk.Label(prediction_window, image=prediction_photo)
        prediction_label.image = prediction_photo
        prediction_label.pack()
        
        
if __name__ == "__main__":
    root = tk.Tk()
    app = ImageAnnotationApp(root)
    root.mainloop()

1.0
27.0 555.0 87.0 850.0
69.05 1419.361111111111 222.49444444444444 2173.796296296296
869.0 1015.0 92.0 216.0
2222.387037037037 2595.7685185185182 235.28148148148148 552.4
934.0 948.0 49.0 57.0
2388.6185185185186 2424.422222222222 125.31296296296296 145.7722222222222
290.0 822.0 12.0 473.0
741.6481481481482 2102.188888888889 30.68888888888889 1209.6537037037037
